## Import des librairies ##

In [28]:
#Imports librairies de base
import numpy as np
import pandas as pd

#Affichage de Graphes
from IPython.display import Markdown, display, HTML
import matplotlib.pyplot as plt

#Information Mutuelle
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif

## 1. Lecture des données ##

1.1 Charger les données d'apprentissage (Data/passagers.csv) dans un DataFrame nommé train. Afficher 10 exemples.

In [4]:
train = pd.read_csv('/content/sample_data/passagers.csv')
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


1.2 Charger les données d'apprentissage (Data/test.csv) dans un Dataframe nommé test. Afficher 10 exemples.

In [7]:
test = pd.read_csv('/content/sample_data/test.csv')
test.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


1.3 Afficher les informations des deux DataFrames train et Test.

In [8]:
print (train.info())
print("--------------------")
print (test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-nul

1.4 Calculer le taux global de survie

In [9]:
survivants = train['Survived'].mean()
print (f"survivants {round(survivants*100, 2)}%")

survivants 38.38%


#### Question 1 : Quel Travail voyez-vous pour que les données soient prêtes à faire tourner dans un modèle ?

1. On peut d'abord commencer par traiter les valeurs manquantes dans les colonnes comme "âge"

2. Traiter les valeurs catégorielles comme pour la colonne Sex (Sex_male, Sex_female)

3. Pour optimiser le temps d'analyse, on peut aussi supprimer des colonnes inutiles.

#### Question 2 : Quelle différence y-a-t'il entre les données train et les données test ?
En se basant sur les resultats de train.info() et test.info() on remarque que:
  - Train comporte a peu près 900 entrées contre 418 pour test. Donc les resultats de données train seront plus précis que test
  - l'absence de la colonne Survived dans test
  - La colonne Embarked dans les données train a quelques valeurs manquantes (889 non-null sur 891 entrées), tandis que dans les données test, elle est complète.

#### Question 3 : Quel est le rôle de ces deux sets de données ?

- Pour les variables de train : Cet ensemble est utilisé pour entraîner le modèle d'apprentissage automatique. Le modèle apprend des modèles et des relations entre les caractéristiques d'entrée (comme la classe, le sexe, l'âge, etc.) et la variable cible (Survived). C'est sur ces données que le modèle ajuste ses paramètres pour minimiser les erreurs de prédiction.

- Pour les variables de test: Cet ensemble est utilisé pour évaluer la performance du modèle après qu'il a été entraîné sur les données d'entraînement. Comme les données de test n'incluent pas la variable cible (Survived), vous utiliserez le modèle entraîné pour prédire la survie sur ces données. Ensuite, ces prédictions sont généralement comparées aux valeurs réelles (qui sont connues mais non utilisées pendant l'entraînement) pour mesurer la précision et la généralisation du modèle à de nouvelles données invisibles


## 2. Analyse des features

2.1 Analyser l'effet de la variable Classe sur le taux de survie, en affichant le % de survie par classe

In [19]:
new_df = train[["Pclass", "Survived"]].dropna()
Taux_survie = new_df.groupby("Pclass")["Survived"].mean()*100
print (Taux_survie)

Pclass
1    62.962963
2    47.282609
3    24.236253
Name: Survived, dtype: float64%


2.2 Analyser l'effet de la variable Genre (Sex) sur le taux de survie, en affichant le % de survie par classe

In [20]:
df_Sex = train[["Sex", "Survived"]].dropna()
Taux_survie_Sex = df_Sex.groupby("Sex")["Survived"].mean()*100
print (Taux_survie_Sex)

Sex
female    74.203822
male      18.890815
Name: Survived, dtype: float64


2.3 Calculer une nouvelle colonne FamilySize contenant la taille de la famille, pour les DataFrame train et Test. Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et la taille de la famille.

In [21]:
train['FamilySize'] = train["SibSp"] + train["Parch"] + 1
test['FamilySize'] = test["SibSp"] + test["Parch"] + 1

print (train[['PassengerId', 'Name', 'Age', 'Sex', 'FamilySize']].head(10))

   PassengerId                                               Name   Age  \
0            1                            Braund, Mr. Owen Harris  22.0   
1            2  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0   
2            3                             Heikkinen, Miss. Laina  26.0   
3            4       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0   
4            5                           Allen, Mr. William Henry  35.0   
5            6                                   Moran, Mr. James   NaN   
6            7                            McCarthy, Mr. Timothy J  54.0   
7            8                     Palsson, Master. Gosta Leonard   2.0   
8            9  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  27.0   
9           10                Nasser, Mrs. Nicholas (Adele Achem)  14.0   

      Sex  FamilySize  
0    male           2  
1  female           2  
2  female           1  
3  female           2  
4    male           1  
5    male           1  
6    m

2.4 Visualisez le nombre de personnes et le taux de survie par taille de famille.

In [22]:
df_FamilySize = train[["FamilySize", "Survived"]].dropna()
Taux_survie_FamilySize = df_FamilySize.groupby("FamilySize")["Survived"].mean()*100
print (Taux_survie_FamilySize)

FamilySize
1     30.353818
2     55.279503
3     57.843137
4     72.413793
5     20.000000
6     13.636364
7     33.333333
8      0.000000
11     0.000000
Name: Survived, dtype: float64


2.5 Calculer une nouvelle colonne IsAlone indiquant si le passager est seul ou pas, pour les DataFrame train et Test. Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et le status IsAlone.

In [25]:
train['IsAlone'] = (train['FamilySize']==1).astype(int) #astype() pour transformer True or False en 1 ou 0
test['IsAlone'] = (test['FamilySize']==1).astype(int)

print (train[['PassengerId', 'Name', 'Age', 'Sex', 'IsAlone']].head(10))

   PassengerId                                               Name   Age  \
0            1                            Braund, Mr. Owen Harris  22.0   
1            2  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0   
2            3                             Heikkinen, Miss. Laina  26.0   
3            4       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0   
4            5                           Allen, Mr. William Henry  35.0   
5            6                                   Moran, Mr. James   NaN   
6            7                            McCarthy, Mr. Timothy J  54.0   
7            8                     Palsson, Master. Gosta Leonard   2.0   
8            9  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  27.0   
9           10                Nasser, Mrs. Nicholas (Adele Achem)  14.0   

      Sex  IsAlone  
0    male        0  
1  female        0  
2  female        1  
3  female        0  
4    male        1  
5    male        1  
6    male        1  
7    m

2.6 Visualisez le nombre de personnes et le taux de survie par statut seul ou pas.

In [26]:
df_IsAlone = train[["IsAlone", "Survived"]].dropna()
Taux_survie_IsAlone = df_IsAlone.groupby("IsAlone")["Survived"].mean()*100
print (Taux_survie_IsAlone)

IsAlone
0    50.564972
1    30.353818
Name: Survived, dtype: float64


2.7 Calculer l'information mutuelle sur les données train.

Attention : Il y a des données continues et des données discrètes

Ignorer les colonnes Name, Ticket et Cabin

In [32]:
train_mi = train.drop(columns=['Name', 'Ticket', 'Cabin'])

# Séparation features / cible
X = train_mi.drop(columns=['Survived'])
y = train_mi['Survived']

# On doit transformer les variables catégorielles en numériques
# Astuce simple : pd.get_dummies
X_encoded = pd.get_dummies(X, drop_first=True)

# Calcul de l'information mutuelle
mi = mutual_info_classif(X_encoded.dropna, y, discrete_features='auto', random_state=42)

# Mise en DataFrame pour affichage plus lisible
mi_df = pd.DataFrame({
    'Feature': X_encoded.columns,
    'MutualInformation': mi
}).sort_values(by='MutualInformation', ascending=False)

# Affichage
print(mi_df)

InvalidParameterError: The 'X' parameter of mutual_info_classif must be an array-like or a sparse matrix. Got <bound method DataFrame.dropna of      PassengerId  Pclass   Age  SibSp  Parch     Fare  FamilySize  IsAlone  \
0              1       3  22.0      1      0   7.2500           2        0   
1              2       1  38.0      1      0  71.2833           2        0   
2              3       3  26.0      0      0   7.9250           1        1   
3              4       1  35.0      1      0  53.1000           2        0   
4              5       3  35.0      0      0   8.0500           1        1   
..           ...     ...   ...    ...    ...      ...         ...      ...   
886          887       2  27.0      0      0  13.0000           1        1   
887          888       1  19.0      0      0  30.0000           1        1   
888          889       3   NaN      1      2  23.4500           4        0   
889          890       1  26.0      0      0  30.0000           1        1  

2.8 Afficher dans un graphique les scores d'information mutuelle par feature

### Question 1 : La variable Classe est-elle utile pour prévoir la survie ? Pourquoi ?



### Question 2 : Même question pour la variable Sex


### Question 3 : Comment traiter le variables liées à la famille du passager ?





## 3. Traitement des données manquantes (à faire sur les données Train et Test)

3.1 La variable "Embarked" possède des valeurs nulles, compléter par la valeur majoritaire (S).

3.2 Afficher l'influence du port d'embarquement sur la Survie

3.3 La variable "Fare" (prix du ticket) possède des valeurs manquantes. Remplacer ces valeurs par la valeur médiane.

3.4 Créer une variable CategoricalFare Découpant la valeur de ticket en 4 catégories de tailles à peu près égales.

3.5 Analyser l'effet de ces catégories de prix de ticket sur la survie.

3.6 Combien de valeurs d'Age manquent ? Complète par des valeurs entières aléatoires entre (moyenne - écart type) et (moyenne + écart type).

3.7 Créer une variable CategoricalAge Découpant la valeur d'age en 5 tranches d'age de même taille.

3.8 Analyser l'effet de ces catégories d'age sur la survie.

### Question 1 : Classer les variables par ordre d'intérêt

## 4 : Traitement de features complexes

4.1 : Le nom permet de retrouver le titre du passager. Trouver les titres lorsqu'ils existent.

4.2 : En utilisant la fonction pandas crosstab, afficher pour train le nombre d'occurences par titres (en ligne) et par sexe (en colonne)

4.3 :
Remplacer 'Melle et 'Ms' par 'Miss'

Remplacer 'Mme' par 'Mrs'

Creer une valeur "Rare" pour les autres titres avec moins de 10 occurences.

Calculer ensuite le taux de survie par titre

## 5. Mise en forme des données non numériques

5.1 Attribuer une valeur numérique au Genre. Par exemple 0 pour 'female' et 1 pour 'male'

5.2 : Attribuer une valeur pour les possibilités de la variable "Title" ('Master','Miss','Mr','Mrs','Rare')

5.3 Remplacer les titres manquant par 0, et attribuer à la colonne le type entier

5.4 Attribuer une valeur numérique aux 4 catégories de la variable Fare

5.5 Attibuer une valeur numérique au Port d'embarquement (Embarked)

5.6 Supprimer les features inutiles

5.7 Afficher les infos pour les deux datasets

5.8 Sauver les deux datasets dans des fichier csv. Eviter d'écrire un index (n° de ligne)